In [1]:
import pandas as pd

In [5]:
df_jan = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
)
df_feb = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"
)

In [6]:
# Q1
df_jan.shape

(3066766, 19)

In [17]:
# Q2
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan["duration"] = df_jan["duration"].apply(lambda td: td.total_seconds() / 60)
df_jan["duration"].std()

42.59435124195458

In [18]:
df_jan["duration"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [52]:
# Q3
df_jan_dropped = df_jan.query("1 <= duration & duration <= 60").copy()
df_jan_dropped.shape[0] / df_jan.shape[0]

0.9812202822125979

In [53]:
# Q4
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
df_jan_dropped[["PULocationID", "DOLocationID"]] = df_jan_dropped[
    ["PULocationID", "DOLocationID"]
].astype(str)
train_dicts = df_jan_dropped[["PULocationID", "DOLocationID"]].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

X_train.shape

(3009173, 515)

In [55]:
# Q5
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

y_train = df_jan_dropped["duration"].values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

np.sqrt(mean_squared_error(y_train, y_pred))

7.64926180044339

In [56]:
# Q6
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration"] = df_feb["duration"].apply(lambda td: td.total_seconds() / 60)
df_feb_dropped = df_feb.query("1 <= duration & duration <= 60").copy()
df_feb_dropped[["PULocationID", "DOLocationID"]] = df_feb_dropped[
    ["PULocationID", "DOLocationID"]
].astype(str)
val_dicts = df_feb_dropped[["PULocationID", "DOLocationID"]].to_dict(orient="records")
X_val = dv.transform(val_dicts)
y_val = df_feb_dropped["duration"].values
y_pred = lr.predict(X_val)
np.sqrt(
    mean_squared_error(
        y_val,
        y_pred,
    )
)

7.8118236307935